In [ ]:
## this code will create a cohort of care home residents 

In [ ]:

library(ggplot2)
library(bigrquery)
library(tidyverse)
library(ggplot2)
library(finalfit)
ProjectId = ""

targetdb1 <-''
targetdb1 <-gsub(' ','',targetdb1)

targetdb2 <-''
targetdb2 <-gsub(' ','',targetdb2)


In [ ]:
## the code in red is the SQL code that I used 

In [ ]:
sql_cohort <- paste('CREATE OR REPLACE TABLE ',targetdb1,'.temp_cb_2172_ch_cohort_master` as (
with a  as (
SELECT person_id, tbl_Services_Start_date, servicetype, FinalisedPSR, row_number() over (partition by person_id order by tbl_services_start_date) as care_seq from ',targetdb2,'.tbl_services
where ServiceType in("Nursing","Residential") AND FORMAT_DATETIME("%d-%b-%Y",tbl_services_start_date) NOT IN  ("04-Apr-2016") AND FORMAT_DATETIME("%d-%b-%Y",tbl_services_start_date) NOT IN ("25-Jul-2016") AND FORMAT_DATETIME("%d-%b-%Y",tbl_services_start_date) NOT IN ("22-Aug-2016")),

b as (
select 
* 
from a
where care_seq = 1
order by person_id, tbl_services_start_date),

c as (SELECT person_id, tbl_Services_end_date, servicetype, row_number() over (partition by person_id order by tbl_services_end_date desc) as care_end_seq from ',targetdb2,'.tbl_services
where ServiceType in("Nursing","Residential") AND FORMAT_DATETIME("%d-%b-%Y",tbl_services_start_date) not in ("04-Apr-2016") AND FORMAT_DATETIME("%d-%b-%Y",tbl_services_start_date) != "25-Jul-2016" AND FORMAT_DATETIME("%d-%b-%Y",tbl_services_start_date) != "22-Aug-2016"),

d as (select
* from c 
where care_end_seq = 1),

e as (
select 
b.person_id,
b.tbl_services_start_date as first_episodestartdate,
d.tbl_services_end_date as last_episodestopdate,
b.ServiceType as admission_service,
b.finalisedPSR
from b 
LEFT join d on
b.person_id= d.person_id
order by person_id),

f as (
Select 
e.*,
datetime_diff(e.last_episodestopdate,e.first_episodestartdate,day) as admission_length
from e
where datetime_diff(e.last_episodestopdate,e.first_episodestartdate,day) > 42),

h as (
SELECT
f.*,
g.birth_datetime,
g.death_datetime,
g.ethnicity_source_value,
g.gender_source_value
FROM f 
LEFT JOIN ',targetdb2,'.person g on
f.person_id = g.person_id)

select
DATE_DIFF(first_episodestartdate, birth_datetime, YEAR) as age_admission,
h.*,
from h
where DATE_DIFF(first_episodestartdate, birth_datetime, YEAR) >= 65 and person_id not in(select person_id from ',targetdb1,'.temp_cb_2172_ch_cohort_missing_data)
order by age_admission desc) ;',sep="")

tbl_cohort <- bq_project_query(ProjectId,sql_cohort)

ch_cohort <- bq_table_download(tbl_cohort)

head(ch_cohort)